# Limpieza de Dataset de Línea de Producción - Versión 2

## Objetivo
Realizar el preprocesamiento y limpieza de un dataset anonimizado de eventos de producción, preparándolo para análisis de eficiencia y minería de texto de comentarios.

## Contexto del Dataset
- **Origen**: Datos de producción anonimizados de una planta industrial
- **Período**: Enero a Diciembre 2024
- **Contenido**:
  - Registros de eventos con marcas temporales
  - Clasificación de incidentes (3 niveles)
  - Comentarios operacionales
  - Turnos codificados

In [2]:
# 1. Configuración Inicial
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
# 2. Carga de Datos
print("[INFO] Cargando dataset anonimizado...")
DATASET_PATH = r"..\data\anonymized-data\efectividad-de-produccion-linea1-2024-encubierta-v2.xlsx"
df = pd.read_excel(DATASET_PATH, engine="openpyxl")

<>:3: SyntaxWarning: invalid escape sequence '\X'
<>:3: SyntaxWarning: invalid escape sequence '\X'
C:\Users\execa\AppData\Local\Temp\ipykernel_6456\832874414.py:3: SyntaxWarning: invalid escape sequence '\X'
  DATASET_PATH = "D:\Xequi 2024\Proyectos_Pesonales\Analisis-Fer-Anonimo\data\Anonymized-data\efectividad-de-produccion-linea1-2024-encubierta-v2.xlsx"


[INFO] Cargando dataset anonimizado...


## Análisis Exploratorio Inicial (EDA)

In [5]:
def resumen_inicial(df):
    """Genera un resumen estructurado del dataset"""
    print("\n[INFO] Resumen inicial del dataset:")
    print(f"\n1. Dimensión del dataset: {df.shape[0]} registros x {df.shape[1]} columnas")
    
    print("\n2. Tipos de datos y valores nulos:")
    print(pd.DataFrame({
        'Tipo': df.dtypes,
        'Valores Nulos': df.isna().sum(),
        '% Nulos': (df.isna().mean()*100).round(2)
    }))
    
    print("\n3. Muestra inicial:")
    display(df.head(3))
    print("\n4. Muestra final:")
    display(df.tail(3))
    
    print("\n5. Rango temporal:")
    print(f"   Inicio: {df['Inicio'].min()}")
    print(f"   Fin: {df['Fin'].max()}")

In [6]:
resumen_inicial(df)


[INFO] Resumen inicial del dataset:

1. Dimensión del dataset: 222403 registros x 10 columnas

2. Tipos de datos y valores nulos:
                       Tipo  Valores Nulos  % Nulos
Inicio       datetime64[ns]              0     0.00
Fin          datetime64[ns]              0     0.00
Duracion            float64              0     0.00
Location             object              0     0.00
Nivel 1              object              0     0.00
Nivel 2              object         202929    91.24
Nivel 3              object         205852    92.56
Comentarios          object         210379    94.59
Team leader          object            819     0.37
Turno                 int64              0     0.00

3. Muestra inicial:


,Inicio,Fin,Duracion,Location,Nivel 1,Nivel 2,Nivel 3,Comentarios,Team leader,Turno
0,2024-01-04 06:30:00.000,2024-01-04 06:30:05.275,5.275,Máquina,Perdida de velocidad,NaN,NaN,NaN,Operario 04,1
1,2024-01-04 06:30:15.036,2024-01-04 06:30:31.862,16.826,Máquina,Paradas cortas,NaN,NaN,NaN,Operario 04,1
2,2024-01-04 06:30:46.502,2024-01-04 06:30:58.058,11.556,Máquina,Paradas cortas,NaN,NaN,NaN,Operario 04,1



4. Muestra final:


,Inicio,Fin,Duracion,Location,Nivel 1,Nivel 2,Nivel 3,Comentarios,Team leader,Turno
222400,2024-12-13 14:30:00,2024-12-13 22:30:00,28800.0,Máquina,El sistema no funciona,Network disconnection,NaN,Network,Operario 02,6
222401,2024-12-13 22:45:00,2024-12-14 06:00:00,26100.0,Máquina,El sistema no funciona,Network disconnection,NaN,Network,NaN,16
222402,2024-12-13 22:45:00,2024-12-14 06:00:00,26100.0,Máquina,El sistema no funciona,Network disconnection,NaN,Network,Operario 03,16



5. Rango temporal:
   Inicio: 2024-01-04 06:30:00
   Fin: 2024-12-14 06:00:00


### Hallazgos Iniciales:
1. **Cobertura Temporal**: 
   - Datos desde 04-Ene-2024 06:30 hasta 14-Dic-2024 06:00
   - Abarca prácticamente todo el año operativo 2024

2. **Estructura de Datos**:
   - Columnas mixtas (datetime, objetos, categóricas)
   - Presencia de valores nulos en columnas de texto

3. **Problemas Detectados**:
   - Inconsistencias en formatos datetime (milisegundos irregulares)
   - Necesidad de estandarización de textos
   - Requiere tratamiento de valores faltantes

## Próximos Pasos:
1. Normalización de formatos temporales
2. Tratamiento de valores nulos
3. Estandarización de textos
4. Validación de rangos lógicos

## 3. Normalización de Datos Temporales

### Objetivo:
Estandarizar el formato de las marcas temporales para garantizar consistencia en los análisis posteriores.

In [7]:
# 3.1 Eliminación de milisegundos inconsistentes
df['Inicio'] = df['Inicio'].dt.floor('s')  # Más eficiente que strftime
df['Fin'] = df['Fin'].dt.floor('s')

# Verificación
print("\nVerificación de tipos de datos:")
print(df[['Inicio', 'Fin']].dtypes)


Verificación de tipos de datos:
Inicio    datetime64[ns]
Fin       datetime64[ns]
dtype: object


### Justificación Técnica:
- `dt.floor('s')` es más eficiente que conversión a string y vuelta a datetime
- Mantiene el tipo datetime64[ns] nativo de pandas
- Elimina inconsistencias en subsegundos

## 4. Manejo de valores faltantes

In [8]:
## 4. Manejo de Valores Faltantes

def reporte_valores_faltantes(df):
    """Genera un reporte detallado de valores nulos"""
    print("\n[ANÁLISIS] Reporte de valores faltantes:")
    null_report = pd.DataFrame({
        'Valores_Nulos': df.isna().sum(),
        'Porcentaje_Nulos': (df.isna().mean()*100).round(2)
    })
    display(null_report)
    
    # Análisis de strings vacíos
    str_cols = df.select_dtypes(include=['object']).columns
    white_spaces = {col: df[col].str.isspace().sum() for col in str_cols}
    print("\nEspacios en blanco:", white_spaces)

reporte_valores_faltantes(df)


[ANÁLISIS] Reporte de valores faltantes:


,Valores_Nulos,Porcentaje_Nulos
Inicio,0,0.00
Fin,0,0.00
Duracion,0,0.00
Location,0,0.00
Nivel 1,0,0.00
Nivel 2,202929,91.24
Nivel 3,205852,92.56
Comentarios,210379,94.59
Team leader,819,0.37
Turno,0,0.00



Espacios en blanco: {'Location': 0, 'Nivel 1': 0, 'Nivel 2': 0, 'Nivel 3': 0, 'Comentarios': 0, 'Team leader': 0}


In [9]:
# 4.1 Tratamiento de valores nulos
print("\n[INFO] Imputando valores faltantes...")
df.fillna('missing', inplace=True)  # Conservamos 'missing' para trazabilidad


[INFO] Imputando valores faltantes...


In [10]:
reporte_valores_faltantes(df)


[ANÁLISIS] Reporte de valores faltantes:


,Valores_Nulos,Porcentaje_Nulos
Inicio,0,0.0
Fin,0,0.0
Duracion,0,0.0
Location,0,0.0
Nivel 1,0,0.0
Nivel 2,0,0.0
Nivel 3,0,0.0
Comentarios,0,0.0
Team leader,0,0.0
Turno,0,0.0



Espacios en blanco: {'Location': 0, 'Nivel 1': 0, 'Nivel 2': 0, 'Nivel 3': 0, 'Comentarios': 0, 'Team leader': 0}


### Estrategia de Tratamiento:
- Campos categóricos: imputación con 'missing' (explícito para análisis).
- Campos temporales/númericos: no requirieron imputación.
- Documentación de decisiones para trazabilidad.

## 5. Consolidación de Registros

In [11]:
# 5.1 Creación de clave de agrupamiento
print("\n[INFO] Creando clave de agrupamiento...")
df['Shift'] = (df[['Nivel 1', 'Nivel 2', 'Nivel 3', 'Comentarios']]
               .ne(df[['Nivel 1', 'Nivel 2', 'Nivel 3', 'Comentarios']].shift())
               .any(axis=1)
               .cumsum())


[INFO] Creando clave de agrupamiento...


In [12]:
# 5.2 Agregación de registros consecutivos
print("[INFO] Consolidando registros...")
agg_rules = {
    'Inicio': 'first',
    'Fin': 'last',
    'Duracion': 'sum',
    'Location': 'first',
    'Nivel 1': 'first',
    'Nivel 2': 'first',
    'Nivel 3': 'first',
    'Comentarios': 'first',
    'Team leader': 'first',
    'Turno': 'first'
}

sintetized_df = df.groupby('Shift', as_index=False).agg(agg_rules)

[INFO] Consolidando registros...


In [13]:
# Validación de integridad
original_duration = df['Duracion'].sum()
consolidated_duration = sintetized_df['Duracion'].sum()
assert abs(original_duration - consolidated_duration) < 1e-6, "Error en consolidación: discrepancia en duraciones"

In [14]:
print(f"\nReducción de dataset: {len(df)} -> {len(sintetized_df)} registros")
print(f"Reducción del {(1-len(sintetized_df)/len(df))*100:.2f}%")


Reducción de dataset: 222403 -> 48495 registros
Reducción del 78.19%


## 6. Enriquecimiento de Datos

### Objetivos:
1. Añadir metadatos temporales y conversiones de unidades para facilitar el análisis.
2. Eliminar columna redundante ('Duracion').

In [15]:
# 6.1 Extracción de componentes temporales
sintetized_df['Mes'] = sintetized_df['Inicio'].dt.month
sintetized_df['Año'] = sintetized_df['Inicio'].dt.year
sintetized_df['Dia_Semana'] = sintetized_df['Inicio'].dt.day_name(locale='es_ES')

In [16]:
# 6.2 Conversión de unidades
sintetized_df['Minutos'] = sintetized_df['Duracion'] / 60
sintetized_df['Horas'] = sintetized_df['Duracion'] / 3600

In [17]:
# 6.3 Reorganización de columnas
column_order = [
    'Inicio', 'Fin', 'Minutos', 'Horas', 
    'Location', 'Nivel 1', 'Nivel 2', 'Nivel 3',
    'Comentarios', 'Team leader', 'Turno',
    'Mes', 'Año', 'Dia_Semana'
]
sintetized_df = sintetized_df[column_order]

### Documentación de Nuevas Columnas:
1. **Mes/Año**: Para análisis temporal agregado
2. **Dia_Semana**: Identificación de patrones semanales
3. **Minutos/Horas**: Unidades más intuitivas para análisis

In [18]:
print("\n[INFO] Estructura final del dataset:")
display(sintetized_df.head(3))
print("\nTipos de datos finales:")
print(sintetized_df.dtypes)


[INFO] Estructura final del dataset:


,Inicio,Fin,Minutos,Horas,Location,Nivel 1,Nivel 2,Nivel 3,Comentarios,Team leader,Turno,Mes,Año,Dia_Semana
0,2024-01-04 06:30:00,2024-01-04 06:30:05,0.087917,0.001465,Máquina,Perdida de velocidad,missing,missing,missing,Operario 04,1,1,2024,Jueves
1,2024-01-04 06:30:15,2024-01-04 06:30:58,0.473033,0.007884,Máquina,Paradas cortas,missing,missing,missing,Operario 04,1,1,2024,Jueves
2,2024-01-04 06:31:17,2024-01-04 06:31:24,0.108100,0.001802,Máquina,Perdida de velocidad,missing,missing,missing,Operario 04,1,1,2024,Jueves



Tipos de datos finales:
Inicio         datetime64[ns]
Fin            datetime64[ns]
Minutos               float64
Horas                 float64
Location               object
Nivel 1                object
Nivel 2                object
Nivel 3                object
Comentarios            object
Team leader            object
Turno                   int64
Mes                     int32
Año                     int32
Dia_Semana             object
dtype: object


In [19]:
# Antes de exportar a MySQL
sintetized_df['Turno'] = sintetized_df['Turno'].astype('object')
sintetized_df['Mes'] = sintetized_df['Mes'].astype('object')
sintetized_df['Año'] = sintetized_df['Año'].astype('object')

# Verificación
print("\n[INFO] Tipos de datos finales:")
print(sintetized_df[['Turno', 'Mes', 'Año']].dtypes)


[INFO] Tipos de datos finales:
Turno    object
Mes      object
Año      object
dtype: object


## 7. Análisis de Relación Temporal entre Eventos (Operaciones Especiales)

### Objetivo:
Identificar y marcar relaciones temporales entre eventos de 'Perdida de velocidad' y otras causas específicas ('Cambio de proceso', 'Fallas del proceso', 'Avería'),
para facilitar el análisis de causalidad y correlación en los procesos productivos.

### Contexto Técnico:
Los eventos de 'perdida de velocidad' son críticos porque:
1. Representan pérdidas de productividad no atribuidas directamente a paradas.
2. Suelen estar relacionados causalmente con otros tipos de eventos.
3. Requieren análisis especial por su naturaleza continua (no son eventos discretos).

In [20]:
# 7.1 Configuración de parámetros
UMBRAL_TIEMPO_SEG = 180  # 3 minutos en segundos
CAUSAS_RELACIONADAS = [
    'Cambio de proceso',      
    'Fallas del proceso',      
    'Averia'                  
]

### Lógica de las Operaciones:
1. Forward Analysis (agregar_comentario):
   - Cuando 'Perdida de velocidad' ocurre <=3min antes de una causa relacionada
   - Marcamos con comentario explicativo

2. Backward Analysis (copiar_valores_anterior):
   - Cuando 'Perdida de velocidad' ocurre <=3min después de cualquier causa
   - Copiamos niveles 2 y 3 de la causa anterior

In [21]:
def detectar_relacion_speedloss(sintetized_df):
    """Realiza análisis temporal de eventos de 'Perdida de velocidad' con otras causas"""
    
    # 7.2 Forward Analysis: 'Perdida de velocidad' -> Causa relacionada
    print("\n[INFO] Ejecutando forward analysis ('Perdida de velocidad' -> Causas)...")
    
    def aplicar_forward(row):
        if row['Nivel 1'] == 'Perdida de velocidad':
            next_idx = row.name + 1
            if next_idx < len(sintetized_df):
                row_next = sintetized_df.iloc[next_idx]
                tiempo_diferencia = (row_next['Inicio'] - row['Fin']).total_seconds()
                if (tiempo_diferencia <= UMBRAL_TIEMPO_SEG and 
                    row_next['Nivel 1'] in CAUSAS_RELACIONADAS):
                    return f"Pérdida de velocidad previo a {row_next['Nivel 1']}"
        return row['Comentarios']
    
    sintetized_df['Comentarios'] = sintetized_df.apply(aplicar_forward, axis=1)
    
    # 7.3 Backward Analysis: Causa -> 'Perdida de velocidad'
    print("[INFO] Ejecutando backward analysis (Causas -> 'Perdida de velocidad')...")
    
    for i in range(1, len(sintetized_df)):
        if (sintetized_df.at[i, 'Nivel 1'] == 'Perdida de velocidad' and
            (sintetized_df.at[i, 'Inicio'] - sintetized_df.at[i-1, 'Fin']).total_seconds() <= UMBRAL_TIEMPO_SEG):
            
            sintetized_df.at[i, 'Nivel 2'] = sintetized_df.at[i-1, 'Nivel 2']
            sintetized_df.at[i, 'Nivel 3'] = sintetized_df.at[i-1, 'Nivel 3']
    
    return sintetized_df

In [22]:
# Ejecutar análisis
sintetized_df = detectar_relacion_speedloss(sintetized_df)


[INFO] Ejecutando forward analysis ('Perdida de velocidad' -> Causas)...
[INFO] Ejecutando backward analysis (Causas -> 'Perdida de velocidad')...


### Resultados Esperados:
1. **Comentarios enriquecidos**: 
   - Registros de 'Perdida de velocidad' que preceden a causas relacionadas, específicamente enumeradas, tendrán comentarios explicativos, señalando esa situación.
   
2. **Causas propagadas**:
   - Registros de 'Perdida de velocidad' que siguen a cualquier causa heredarán sus niveles 2 y 3.
   - Permite rastrear el origen técnico de la pérdida de velocidad.

### Validación:
Se recomienda generar métricas post-ejecución:
1. Conteo de relaciones 'Perdida de velocidad'-> Causa detectadas.
2. Conteo de causas propagadas a 'Perdida de velocidad'.
3. Distribución temporal de estos eventos.

In [23]:
def generar_reporte_relaciones(df):  # 'df' como parámetro genérico
    forward = df['Comentarios'].str.contains('Pérdida de velocidad previo a').sum()
    backward = sum(
        (df['Nivel 1'] == 'Perdida de velocidad') & 
        (df['Nivel 2'] != df['Nivel 2'].shift(1))
    )
    
    print(f"\n[REPORTE] Relaciones detectadas:")
    print(f"- 'Perdida de velocidad' -> Causas relacionadas: {forward} casos")
    print(f"- Causas -> 'Perdida de velocidad' (herencia): {backward} casos")

In [24]:
# Llamada con sintetized_df
generar_reporte_relaciones(sintetized_df)


[REPORTE] Relaciones detectadas:
- 'Perdida de velocidad' -> Causas relacionadas: 804 casos
- Causas -> 'Perdida de velocidad' (herencia): 94 casos


## 8. Exportación de Resultados y Cierre de la primera etapa del Proyecto

### Objetivo:
Persistir los datos procesados en formatos adecuados para:
1. Análisis en herramientas de BI.
2. Integración con sistemas existentes.
3. Desarrollo de aplicaciones ejecutables.

In [ ]:
# 8.1 Configuración de rutas y parámetros
import os
from datetime import datetime

# Configuración de exportación - RUTA PERSONALIZADA

RUTA_SALIDA = r"..\data\cleaned-data"
NOMBRE_BASE = "efectividad-de-produccion-linea1-2024-procesada"
VERSION = datetime.now().strftime('%Y%m%d_%H%M')
EXTENSION = ".xlsx"

ARCHIVO_COMPLETO = os.path.join(RUTA_SALIDA, f"{NOMBRE_BASE}_{VERSION}{EXTENSION}")

# Crear directorio si no existe
os.makedirs(RUTA_SALIDA, exist_ok=True)

In [26]:
# 8.2 Exportación a Excel (para BI)
print("\n[INFO] Exportando datos a Excel...")
sintetized_df.to_excel(ARCHIVO_COMPLETO, index=False, engine='openpyxl')
print(f"✔ Datos exportados a: {ARCHIVO_COMPLETO}")


[INFO] Exportando datos a Excel...
✔ Datos exportados a: D:\Xequi 2024\Proyectos_Pesonales\Analisis-Fer-Anonimo\data\data-cleaned\efectividad-de-produccion-linea1-2024-procesada_20250331_1833.xlsx


In [30]:
# 8.3 Exportación a MySQL (para integración)

from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from sqlalchemy.types import DateTime, Float, String
import sys

# Cargar variables
load_dotenv()  # Busca automáticamente el archivo .env

print("\n[INFO] Conectando a base de datos MySQL...")
try:
    
    # Verificar variables de entorno
    required_vars = ['DB_USER', 'DB_PASSWORD', 'DB_HOST', 'DB_PORT', 'DB_NAME']
    missing_vars = [var for var in required_vars if not os.getenv(var)]
    
    if missing_vars:
        raise ValueError(f"Variables de entorno faltantes: {', '.join(missing_vars)}")
       
    # Configuración de tipos
    schema = {
        'Inicio': DateTime(),
        'Fin': DateTime(),
        'Minutos': Float(),
        'Horas': Float(),
        'Location': String(45),
        'Nivel 1': String(125),  # Nota: Usar el nombre original de la columna
        'Nivel 2': String(125),
        'Nivel 3': String(125),
        'Comentarios': String(500),
        'Team leader': String(45),  # Asegurar matching con nombres de columnas
        'Turno': String(45),
        'Mes': String(45),
        'Año': String(45),
        'Dia_Semana': String(45)
    }
    
    # Conexión
    engine = create_engine(
        f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
        f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}",
        connect_args={
            'ssl': {'ssl_mode': 'preferred'}  # Opcional para conexiones seguras
        },
        pool_pre_ping=True  # Prueba la conexión antes de usarla
    )
    
    # Test de conexión
    with engine.connect() as conn:
        print("✔ Conexión a MySQL verificada")
    
    # Exportación con configuración segura
    tabla_mysql = 'linea_1'
    
    print(f"[INFO] Subiendo datos a tabla {tabla_mysql}...")
    sintetized_df.to_sql(
        name=tabla_mysql,
        con=engine,
        if_exists='append',  # Correcto para tu caso de uso
        index=False,
        dtype=schema,
        chunksize=1000,  # Mejor rendimiento para grandes volúmenes
        method='multi'  # Insersión más eficiente
    )
    print("✔ Datos cargados en MySQL correctamente")
    
except ImportError as e:
    print(f"✖ Error: Paquete requerido faltante. Ejecuta: pip install {e.name}", file=sys.stderr)
    print("⚠ Continuando sin exportación a base de datos")

except ValueError as e:
    print(f"✖ Error de configuración: {str(e)}", file=sys.stderr)
    print("⚠ Verifica tu archivo .env", file=sys.stderr)

except Exception as e:
    print(f"✖ Error en conexión MySQL: {str(e)}", file=sys.stderr)
    print("⚠ Continuando sin exportación a base de datos")


[INFO] Conectando a base de datos MySQL...
✔ Conexión a MySQL verificada
[INFO] Subiendo datos a tabla linea_1...
✔ Datos cargados en MySQL correctamente


## 9. Documentación Final y Próximos Pasos

### Resultados Obtenidos:
1. Dataset anonimizado y procesado disponible en:
   - Excel: para análisis en herramientas de BI (PowerBI, Tableau)
   - MySQL: para integración con otros sistemas

2. Metadatos generados:
   - Estructura de eventos temporal relacionada
   - Clasificación estandarizada de causas
   - Comentarios enriquecidos

### Próximos Pasos Recomendados:

1. Desarrollo de Interfaz Ejecutable:
   - Usar PyInstaller: `pyinstaller --onefile --windowed procesador_datos.py`
   - Incluir archivos de configuración:
     - `config_db.py` para conexiones
     - `config_params.py` para parámetros de procesamiento

2. Análisis de Series Temporales:
   ```python
   from statsmodels.tsa.seasonal import seasonal_decompose
   # Ejemplo básico de análisis
   serie_temporal = sintetized_df.groupby(pd.Grouper(key='Inicio', freq='D'))['Minutos'].sum()
   decomposition = seasonal_decompose(serie_temporal, model='additive')
   decomposition.plot()

3. Proyectos SQL Complementarios:
    - Vistas materializadas para reportes frecuentes
    - Procedimientos almacenados para ETL periódico
    - Consultas optimizadas para dashboards